# 16.어텐션 메커니즘 (Attention Mechanism)

이번 챕터에서는 신경망들의 성능을 높이기 위한 메커니즘이자, 이제는 AI 분야에서 대세 모듈로서 사용되고 있는 트랜스포머의 기반이 되는 어텐션 메커니즘에 대해서 학습한다.


## 1) 어텐션 메커니즘 (Attention Mechanism)

앞서 배운 seq2seq 모델은 **인코더**에서 입력 시퀀스를 컨텍스트 벡터라는 하나의 고정된 크기의 벡터 표현으로 압축하고, **디코더**는 이 컨텍스트 벡터를 통해서 출력 시퀀스를 만들어냈다.

하지만 이러한 RNN에 기반한 seq2seq 모델에는 크게 두 가지 문제가 있다.

* **첫째, 하나의 고정된 크기의 벡터에 모든 정보를 압축하려고 하니까 정보 손실이 발생한다.**
* **둘째, RNN의 고질적인 문제인 기울기 소실(Vanishing Gradient) 문제가 존재한다.**

즉, 결국 이는 기계 번역 분야에서 입력 문장이 길면 번역 품질이 떨어지는 현상으로 나타났다. 이를 위한 대안으로 입력 시퀀스가 길어지면 출력 시퀀스의 정확도가 떨어지는 것을 보정해주기 위해 등장한 기법인 어텐션(attention)을 소개한다.



### 1.어텐션(Attention)의 아이디어

어텐션의 기본 아이디어는 디코더에서 출력 단어를 예측하는 매 시점(timestep)마다, 인코더에서의 전체 입력 문장을 다시 한 번 참고한다는 점이다. 단, 전체 입력 문장을 전부 다 동일한 비율로 참고하는 것이 아니라, 해당 시점에서 예측해야 할 단어와 연관이 있는 입력 단어 부분을 좀 더 집중(attention)해서 보게 된다.



### 2.어텐션 함수(Attention Function)

어텐션 메커니즘을 언급하기 전에 컴퓨터공학의 많은 분야에서 사용되는 Key-Value로 구성되는 자료형에 대해서 잠깐 언급하겠다. 가령, 이 책의 주 언어로 사용되는 파이썬에도 Key-Value로 구성되는 자료형인 딕셔너리(Dict) 자료형이 존재한다. 파이썬의 딕셔너리 자료형은 키(Key)와 값(Value)이라는 두 개의 쌍으로 구성되는데, 키를 통해서 맵핑된 값을 찾아낼 수 있다는 특징을 갖고 있다.



In [1]:
# 파이썬의 딕셔너리 자료형을 선언
# 키(Key) : 값(Value)의 형식으로 키와 값의 쌍(Pair)을 선언한다.
Dict = {"2017" : "Transformer", "2018" : "BERT"}

위의 자료형에서 2017은 키에 해당되며, Transformer는 2017의 키와 맵핑되는 값에 해당된다. 그와 마찬가지로 2018은 키에 해당되며, BERT는 2018이라는 키와 맵핑되는 값에 해당된다.


In [2]:
print(Dict['2017']) # 2017이라는 키에 해당되는 값을 출력

Transformer


In [3]:
print(Dict['2018']) # 2018이라는 키에 해당되는 값을 출력

BERT


Key-Value 자료형에 대한 이해를 가지고, 어텐션 함수에 대해서 설명해보겠다.

<img src = 'https://wikidocs.net/images/page/22893/%EC%BF%BC%EB%A6%AC.PNG' width = 60%>

어텐션을 함수로 표현하면 주로 다음과 같이 표현된다.

**Attention(Q, K, V) = Attention Value**

어텐션 함수는 주어진 '쿼리(Query)'에 대해서 모든 '키(Key)'와의 유사도를 각각 구한다. 그리고 구해낸 이 유사도를 키와 맵핑되어있는 각각의 '값(Value)'에 반영해준다. 그리고 유사도가 반영된 '값(Value)'을 모두 더해서 리턴한다. 여기서는 이를 어텐션 값(Attention Value)이라고 하겠다.

지금부터 배우게 되는 seq2seq + 어텐션 모델에서 Q, K, V에 해당되는 각각의 Query, Keys, Values는 각각 다음과 같다.

* Q = Query : t 시점의 디코더 셀에서의 은닉 상태
* K = Keys : 모든 시점의 인코더 셀의 은닉 상태들
* V = Values : 모든 시점의 인코더 셀의 은닉 상태들

이제 매우 간소화된 어텐션 예제를 통해 어텐션을 이해해보도록 하겠다.


### 3.닷-프로덕트 어텐션(Dot-Product Attention)

어텐션은 다양한 종류가 있는데 그 중에서도 가장 수식적으로 이해하기 쉽게 수식을 적용한 닷-프로덕트 어텐션(Dot-Product Attention)을 통해 어텐션을 이해해보도록 하겠다. seq2seq에서 사용되는 어텐션 중에서 닷-프로덕트 어텐션과 다른 어텐션의 차이는 주로 중간 수식의 차이로 메커니즘 자체는 거의 유사하다.

<img src = 'https://wikidocs.net/images/page/22893/dotproductattention1_final.PNG' width = 60%>



위 그림은 디코더의 세번째 LSTM 셀에서 출력 단어를 예측할 때, 어텐션 메커니즘을 사용하는 모습을 보여준다. 디코더의 첫번째, 두번째 LSTM 셀은 이미 어텐션 메커니즘을 통해 je와 suis를 예측하는 과정을 거쳤다고 가정한다. 어텐션 메커니즘에 대해 상세히 설명하기 전에 위의 그림을 통해 전체적인 감만 우선 잡고 들어가보겠다. 디코더의 세번째 LSTM 셀은 출력 단어를 예측하기 위해서 인코더의 모든 입력 단어들의 정보를 다시 한 번 참고하고자 한다. 중간 과정에 대한 설명은 현재는 생략하고 여기서 주목할 것은 인코더의 소프트맥스 함수이다.

소프트맥스 함수를 통해 나온 결과값은 I, am, a, student 단어 각각이 출력 단어를 예측할 때 얼마나 도움이 되는지의 정도를 수치화한 값이다. 위의 그림에서는 빨간 직사각형의 크기로 소프트맥스 함수의 결과값의 크기를 표현했다. 직사각형의 크기가 클수록 도움이 되는 정도의 크기가 크다. 각 입력 단어가 디코더의 예측에 도움이 되는 정도가 수치화하여 측정되면 이를 하나의 정보로 담아서 디코더로 전송된다. 위의 그림에서는 초록색 삼각형이 이에 해당된다. 결과적으로, 디코더는 출력 단어를 더 정확하게 예측할 확률이 높아진다. 이제 어텐션 메커니즘에 대한 전체적인 감을 잡았으면 어텐션 메커니즘에 대해 상세히 알아보겠다.



#### 1) 어텐션 스코어(Attention Score)를 구한다.

<img src = 'https://wikidocs.net/images/page/22893/dotproductattention2_final.PNG' width = 60%>

인코더의 시점(time step)을 각각 1, 2, ..., N이라고 하였을 때 인코더의 은닉 상태(hidden state)를 각각 $h_{1}, h_{2}, ..., h_{N}$이라고 하자. 디코더의 현재 시점(time step) t에서의 디코더의 은닉 상태(hidden state)를 $s_{t}$라고 하자. 또한 여기서는 인코더의 은닉 상태와 디코더의 은닉 상태의 차원이 같다고 가정한다. 위의 그림의 경우에는 인코더의 은닉 상태와 디코더의 은닉 상태가 동일하게 차원이 4이다.

어텐션 메커니즘의 첫 걸음인 어텐션 스코어(Attention score)에 대해서 배우기 전에, 이전 챕터에서 배웠던 디코더의 현재 시점 t에서 필요한 입력값을 다시 상기해보겠다. 시점 t에서 출력 단어를 예측하기 위해서 디코더의 셀은 두 개의 입력값을 필요로 하는데, 바로 이전 시점인 t-1의 은닉 상태와 이전 시점 t-1에 나온 출력 단어이다.

그런데 어텐션 메커니즘에서는 출력 단어 예측에 또 다른 값을 필요로 하는데 바로 어텐션 값(Attention Value)이라는 새로운 값이다. t번째 단어를 예측하기 위한 어텐션 값을 $a_{t}$라고 정의하겠다.

어텐션 값이라는 새로운 개념이 등장한 만큼, 어텐션 값이 현재 시점 t에서의 출력 예측에 구체적으로 어떻게 반영되는지는 뒤에서 설명하겠다. 지금부터 배우는 모든 과정은 $a_{t}$를 구하기 위한 여정이다. 그리고 그 여정의 첫 걸음은 바로 어텐션 스코어(Attention Score)를 구하는 일이다. 어텐션 스코어란 현재 디코더의 시점 t에서 단어를 예측하기 위해, 인코더의 모든 은닉 상태 각각이 디코더의 현 시점의 은닉 상태 $s_{t}$와 얼마나 유사한지를 판단하는 스코어값이다.

닷-프로덕트 어텐션에서는 이 스코어 값을 구하기 위해 $s_{t}$를 전치(transpose)하고 각 은닉 상태와 내적(dot product)을 수행한다. 즉, 모든 어텐션 스코어 값은 스칼라이다. 예를 들어 $s_{t}$와 인코더의 i번째 은닉 상태의 어텐션 스코어의 계산 방법은 아래와 같다.

<img src = 'https://wikidocs.net/images/page/22893/i%EB%B2%88%EC%A7%B8%EC%96%B4%ED%85%90%EC%85%98%EC%8A%A4%EC%BD%94%EC%96%B4_final.PNG' width = 60%>

어텐션 스코어 함수를 정의해보면 다음과 같다.

$score(s_{t},\ h_{i}) = s_{t}^Th_{i}$

$s_{t}$와 인코더의 모든 은닉 상태의 어텐션 스코어의 모음값을 $e^{t}$라고 정의하겠다. $e^{t}$의 수식은 다음과 같다.

$e^{t}=[s_{t}^Th_{1},...,s_{t}^Th_{N}]$



#### 2) 소프트맥스(softmax) 함수를 통해 어텐션 분포(Attention Distribution)를 구한다.

<img src = 'https://wikidocs.net/images/page/22893/dotproductattention3_final.PNG' width = 60%>

$e^{t}$에 소프트맥스 함수를 적용하여, 모든 값을 합하면 1이 되는 확률 분포를 얻어낸다. 이를 어텐션 분포(Attention Distribution)라고 하며, 각각의 값은 어텐션 가중치(Attention Weight)라고 한다. 예를 들어 소프트맥스 함수를 적용하여 얻은 출력값인 I, am, a, student의 어텐션 가중치를 각각 0.1, 0.4, 0.1, 0.4라고 하자. 이들의 합은 1이다. 위의 그림은 각 인코더의 은닉 상태에서의 어텐션 가중치의 크기를 직사각형의 크기를 통해 시각화하였다. 즉, 어텐션 가중치가 클수록 직사각형이 크다.

디코더의 시점 t에서의 어텐션 가중치의 모음값인 어텐션 분포를 $α^{t}$이라고 할 때, $α^{t}$를 식으로 정의하면 다음과 같다.

$α^{t} = softmax(e^{t})$


#### 3) 각 인코더의 어텐션 가중치와 은닉 상태를 가중합하여 어텐션 값(Attention Value)을 구한다.

<img src = 'https://wikidocs.net/images/page/22893/dotproductattention4_final.PNG' width = 60%>

이제 지금까지 준비해온 정보들을 하나로 합치는 단계이다. 어텐션의 최종 결과값을 얻기 위해서 각 인코더의 은닉 상태와 어텐션 가중치값들을 곱하고, 최종적으로 모두 더한다. 요약하면 가중합(Weighted Sum)을 한다고 말할 수도 있겠다. 아래는 어텐션의 최종 결과. 즉, 어텐션 함수의 출력값인 어텐션 값(Attention Value) $a_{t}$에 대한 식을 보여준다.

$a_{t}=\sum_{i=1}^{N} α_{i}^{t}h_{i}$

이러한 어텐션 값 $a_{t}$는 종종 인코더의 문맥을 포함하고 있다고하여, **컨텍스트 벡터(context vector)**라고도 불린다. 앞서 배운 가장 기본적인 seq2seq에서는 인코더의 마지막 은닉 상태를 컨텍스트 벡터라고 부르는 것과 대조된다.



#### 4) 어텐션 값과 디코더의 t 시점의 은닉 상태를 연결한다.(Concatenate)

<img src = 'https://wikidocs.net/images/page/22893/dotproductattention5_final_final.PNG' width = 60%>

이제 어텐션 함수의 최종값인 어텐션 값 $a_{t}$를 구했다. 앞서 어텐션 메커니즘이 들어간 t시점의 은닉 상태를 구하는 방법의 식으로 다음과 같은 식을 소개한 바 있다. 사실 어텐션 값이 구해지면 어텐션 메커니즘은 $a_{t}$를 $s_{t}$와 결합(concatenate)하여 하나의 벡터로 만드는 작업을 수행한다. 이를 $v_{t}$라고 정의해보겠다. 그리고 이 $v_{t}$를 $\hat{y}$ 예측 연산의 입력으로 사용함으로써 인코더로부터 얻은 정보를 활용하여 $\hat{y}$을 좀 더 잘 예측할 수 있게 된다. 이것이 어텐션 메커니즘의 핵심이다.



#### 5) 출력층 연산의 입력이 되는 $\tilde{{s}}_{t}$를 계산한다.

<img src = 'https://wikidocs.net/images/page/22893/st.PNG' width = 60%>

논문에서는 $v_{t}$를 바로 출력층으로 보내기 전에 신경망 연산을 한 번 더 추가하였다. 가중치 행렬과 곱한 후에 하이퍼볼릭탄젠트 함수를 지나도록 하여 출력층 연산을 위한 새로운 벡터인 $\tilde{{s}}_{t}$를 얻는다. 어텐션 메커니즘을 사용하지 않는 seq2seq에서는 출력층의 입력이 t시점의 은닉 상태인 ${s}_{t}$였던 반면, 어텐션 메커니즘에서는 출력층의 입력이 $\tilde{{s}}_{t}$가 되는 셈이다.

이를 식으로 표현하면 다음과 같다. 식에서 $\mathbf{W_{c}}$는 학습 가능한 가중치 행렬, $b_{c}$는 편향이다. 그림에서 편향은 생략했다.

$\tilde{s}_{t} = \tanh(\mathbf{W_{c}}[{α}_t;{s}_t] + b_{c})$



#### 6) $\tilde{{s}}_{t}$를 출력층의 입력으로 사용한다.

$\tilde{{s}}_{t}$를 출력층의 입력으로 사용하여 예측 벡터를 얻는다.

$\widehat{y}_t = \text{Softmax}\left( W_y\tilde{s}_t + b_y \right)$



### 4.다양한 종류의 어텐션(Attention)

앞서 seq2seq + 어텐션(attention) 모델에 쓰일 수 있는 다양한 어텐션 종류가 있지만, 닷-프로덕트 어텐션과 다른 어텐션들의 차이는 중간 수식의 차이라고 언급한 바 있다. 여기서 말하는 중간 수식은 어텐션 스코어 함수를 말한다. 위에서 배운 어텐션이 닷-프로덕트 어텐션인 이유는 어텐션 스코어를 구하는 방법이 내적이었기 때문이다.

어텐션 스코어를 구하는 방법은 여러가지가 제시되어 있으며, 현재 제시된 여러 종류의 어텐션 스코어 함수는 다음과 같다.


|**이름**	|**스코어 함수**	|**Defined by**
|--|--|--	
|$dot$	|$score(s_{t},\ h_{i}) = s^{T}_{t}h_{i}$|Luong et al. (2015)	
|$scaled$ $dot$|$score(s_{t},\ h_{i}) = \frac{s^{T}_{t}h_{i}}{\sqrt{n}}$|Vaswani et al. (2017)	
|$general$|$score(s_{t},\ h_{i}) = s^{T}_{t}W_{a}h_{i}$// 단, $W_{a}$는 학습 가능한 가중치 행렬|Luong et al. (2015)	
|$concat$|$score(s_{t},\ h_{i}) = W_{a}^{T}\ tanh(W_{b}[s_{t};h_{i}]) \\ score(s_{t},\  h_{i}) = W_{a}^{T}\ tanh(W_{b}s_{t}+W_{c} h_{i})$|Bahdanau et al. (2015)	
|$location−base$|$α_{t} = softmax(W_{a}s_{t})$// $α_{t}$ 산출 시에 $s_{t}$만 사용하는 방법.|Luong et al. (2015)

위에서 $s_{t}$는 Query, $h_{i}$는 Keys, $W_{a}$와 $W_{b}$는 학습 가능한 가중치 행렬이다.

이름이 dot이라고 붙여진 스코어 함수가 이번 챕터에서 배운 닷 프로덕트 어텐션이다. 이 어텐션은 제안한 사람의 이름을 따서 루옹(Luong) 어텐션이라고도 한다. 제안한 이들의 이름은 위 테이블에서 Defined By에 적혀져 있다. 그렇다면, concat이라는 이름의 어텐션은 만든 사람의 이름을 따서 바다나우(Bahdanau) 어텐션이라고도 부를 것이다.

지금까지 seq2seq에서 성능을 향상시켜주기 위한 기법인 어텐션에 대해서 알아보았다. 어텐션은 처음에는 RNN 기반의 seq2seq의 성능을 보정하기 위한 목적으로 소개되었지만, 현재에 이르러서는 어텐션 스스로가 기존의 seq2seq를 대체하는 방법이 되어가고 있다. 이에 대해서는 다음 챕터인 트랜스포머(Transformer) 챕터에서 더 자세히 배워보도록 하겠다.



어텐션을 소개한 논문 : https://arxiv.org/pdf/1409.0473.pdf

추천하는 참고 자료 : http://docs.likejazz.com/attention/

https://www.cs.cmu.edu/~bhiksha/courses/deeplearning/Fall.2015/slides/lec14.neubig.seq_to_seq.pdf

https://lilianweng.github.io/lil-log/2018/06/24/attention-attention.html

https://towardsdatascience.com/attn-illustrated-attention-5ec4ad276ee3